In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

In [24]:
def CategoricalConverter(column):
    onehotencoder = OneHotEncoder()
    prod = onehotencoder.fit_transform(pd.DataFrame(column)).toarray() # Convertir a categórico
    prod = pd.DataFrame(prod, columns = np.sort(column.unique())) #Hacer un df con columnas
    return prod

In [25]:
trs = pd.read_csv('../Test Data/Transaccion_test.csv')
trs.drop(['Unnamed: 0' , 'Id_Producto' , 'Tipo' , 'Fecha'] , axis = 1 , inplace = True)

In [26]:
trs['Signo'] = trs['Signo'].fillna("Positivo")

In [27]:
trs

,id,Producto-Tipo,Signo,Monto,Periodo
0,1,F-D,Positivo,405714.281250,202008
1,1,F-D,Positivo,591428.562500,202009
2,1,F-D,Positivo,405714.281250,202010
3,1,F-H,Negativo,405714.281250,202008
4,1,F-H,Negativo,591428.562500,202009
...,...,...,...,...,...
3743640,79539,G-K,Negativo,123282.859375,202010
3743641,79539,G-K,Negativo,122348.570312,202010
3743642,79539,G-K,Negativo,165910.421875,202010
3743643,79539,G-K,Negativo,121699.429688,202010


In [28]:
trsfinal = trs.copy()
trsfinal.set_index('id' , inplace = True)
trsfinal.dropna(inplace = True) #se eliminaron 1.439.192 filas

In [30]:
trsfinal.reset_index(inplace = True)

In [31]:
signo = trsfinal['Signo'].values
montoSigno = []
for i in signo:
    if i == 'Positivo':
        montoSigno.append(1)
    else:
        montoSigno.append(-1)
montoSigno = np.array(montoSigno)
monto = trsfinal['Monto'].values
trsfinal['MontoSigno'] = np.multiply(monto,montoSigno) # Multiplicación
trsfinal.drop(['Signo' , 'Monto'] , axis = 1 , inplace = True)
trsfinal.reset_index(drop = True , inplace = True)

In [32]:
trsfinal

,id,Producto-Tipo,Periodo,MontoSigno
0,1,F-D,202008,405714.281250
1,1,F-D,202009,591428.562500
2,1,F-D,202010,405714.281250
3,1,F-H,202008,-405714.281250
4,1,F-H,202009,-591428.562500
...,...,...,...,...
3743640,79539,G-K,202010,-123282.859375
3743641,79539,G-K,202010,-122348.570312
3743642,79539,G-K,202010,-165910.421875
3743643,79539,G-K,202010,-121699.429688


# Pivot Table

In [33]:
pivot2 = pd.pivot_table(trsfinal , values = 'MontoSigno' , index = ['id'] , columns = 'Producto-Tipo' , fill_value = 0 , aggfunc = np.sum)

In [34]:
pivot2.reset_index(inplace = True)

In [35]:
pivot2

Producto-Tipo,id,A-G,A-K,A-L,C-C,D-F,E-F,F-D,F-H,F-I,F-J,G-K
0,1,1149213,-4.900651e+06,0.0000,0.000000e+00,0,0,1.402857e+06,-1.402857e+06,-1402857.125,1.402857e+06,-2.690158e+06
1,2,1113246,-2.996131e+06,0.0000,-3.694527e+06,-121141,0,0.000000e+00,-3.600000e+05,0.000,3.600000e+05,-7.671284e+06
2,3,2510778,-4.986702e+06,0.0000,0.000000e+00,0,-123745,1.200361e+05,-3.600361e+05,0.000,3.600361e+05,-6.503626e+05
3,4,1122183,-1.001903e+06,548571.4375,0.000000e+00,0,-121428,0.000000e+00,-1.200000e+05,-240000.000,3.600000e+05,-2.947002e+06
4,5,494103,-6.065471e+05,0.0000,0.000000e+00,0,0,0.000000e+00,-3.600000e+05,-120000.000,3.600000e+05,-4.998845e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
79534,79535,678924,-1.171770e+06,0.0000,-6.029971e+05,-122047,0,0.000000e+00,-2.400000e+05,-360000.000,3.600000e+05,-7.566180e+06
79535,79536,713488,-3.674494e+05,0.0000,0.000000e+00,0,0,1.270000e+05,-1.200000e+05,-247000.000,2.470000e+05,-5.894302e+06
79536,79537,568955,-2.513046e+05,0.0000,0.000000e+00,0,-150370,1.028571e+06,-1.200000e+05,0.000,1.148571e+06,0.000000e+00
79537,79538,767589,-2.220285e+06,0.0000,0.000000e+00,0,0,0.000000e+00,-3.600000e+05,-360000.000,3.600000e+05,-2.487990e+05


In [36]:
pivot3 = pivot2.copy()

In [37]:
pivot3

Producto-Tipo,id,A-G,A-K,A-L,C-C,D-F,E-F,F-D,F-H,F-I,F-J,G-K
0,1,1149213,-4.900651e+06,0.0000,0.000000e+00,0,0,1.402857e+06,-1.402857e+06,-1402857.125,1.402857e+06,-2.690158e+06
1,2,1113246,-2.996131e+06,0.0000,-3.694527e+06,-121141,0,0.000000e+00,-3.600000e+05,0.000,3.600000e+05,-7.671284e+06
2,3,2510778,-4.986702e+06,0.0000,0.000000e+00,0,-123745,1.200361e+05,-3.600361e+05,0.000,3.600361e+05,-6.503626e+05
3,4,1122183,-1.001903e+06,548571.4375,0.000000e+00,0,-121428,0.000000e+00,-1.200000e+05,-240000.000,3.600000e+05,-2.947002e+06
4,5,494103,-6.065471e+05,0.0000,0.000000e+00,0,0,0.000000e+00,-3.600000e+05,-120000.000,3.600000e+05,-4.998845e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
79534,79535,678924,-1.171770e+06,0.0000,-6.029971e+05,-122047,0,0.000000e+00,-2.400000e+05,-360000.000,3.600000e+05,-7.566180e+06
79535,79536,713488,-3.674494e+05,0.0000,0.000000e+00,0,0,1.270000e+05,-1.200000e+05,-247000.000,2.470000e+05,-5.894302e+06
79536,79537,568955,-2.513046e+05,0.0000,0.000000e+00,0,-150370,1.028571e+06,-1.200000e+05,0.000,1.148571e+06,0.000000e+00
79537,79538,767589,-2.220285e+06,0.0000,0.000000e+00,0,0,0.000000e+00,-3.600000e+05,-360000.000,3.600000e+05,-2.487990e+05


In [38]:
pivot3.to_pickle('TestDF.pkl')

In [28]:
pivotFinal = pivot3.copy()

In [29]:
pivotFinal.set_index('id' , inplace = True)

In [30]:
pivotFinal

Producto-Tipo,A-G,A-K,A-L,C-C,D-F,E-F,F-D,F-H,F-I,F-J,G-K
id,,,,,,,,,,,
1,4953988,-3.686182e+07,0.000000e+00,0.000000e+00,0,0,7.321078e+06,-7.561078e+06,-1.549649e+06,7.561078e+06,-5.770217e+06
2,8433301,-2.367750e+07,0.000000e+00,-1.605180e+07,-1699501,-384461,2.664929e+07,-1.151481e+06,-9.371714e+06,2.784929e+07,-4.938157e+07
3,10501637,-3.556323e+07,1.120000e+06,0.000000e+00,0,-1367568,1.802073e+06,-2.282073e+06,-6.012641e+05,2.282073e+06,-2.034374e+07
4,3696143,-9.662305e+06,1.697867e+05,0.000000e+00,-491115,-618740,6.297714e+07,-6.345714e+07,-1.440000e+06,6.513714e+07,-2.362493e+07
5,2327401,-9.319559e+06,0.000000e+00,0.000000e+00,-243570,-363838,2.811429e+06,-4.611429e+06,-4.491429e+06,4.611429e+06,-3.446029e+07
...,...,...,...,...,...,...,...,...,...,...,...
79535,3161029,-1.767045e+07,0.000000e+00,0.000000e+00,-854239,-240947,2.292857e+06,-2.115000e+06,-3.372857e+06,3.492857e+06,-3.040691e+07
79536,3621677,-1.738624e+07,0.000000e+00,0.000000e+00,0,0,2.034100e+06,-1.539227e+06,-3.234100e+06,2.971243e+06,-3.196310e+07
79537,130260,-2.513137e+05,0.000000e+00,-8.082290e+06,0,-495712,8.725352e+06,-3.198913e+06,0.000000e+00,8.965352e+06,0.000000e+00


# TargetDf

In [31]:
targetdf.head()

,id,Producto-Tipo,Signo,Monto,Periodo
0,3,A-A,Positivo,234285.71875,201906
1,4,A-A,Positivo,362857.15625,201912
2,6,A-A,Positivo,220000.00000,201901
3,6,A-A,Positivo,220000.00000,201904
4,7,A-A,Positivo,220000.00000,201903


In [32]:
targetdf2 = targetdf.copy()
signo = targetdf2['Signo'].values
montoSigno = []
for i in signo:
    if i == 'Positivo':
        montoSigno.append(1)
    else:
        montoSigno.append(-1)
montoSigno = np.array(montoSigno)
monto = targetdf2['Monto'].values
targetdf2['MontoSigno'] = np.multiply(monto,montoSigno) # Multiplicación
targetdf2.drop(['Signo' , 'Monto'] , axis = 1 , inplace = True)
targetdf2.reset_index(drop = True , inplace = True)

In [33]:
targetdf2

,id,Producto-Tipo,Periodo,MontoSigno
0,3,A-A,201906,234285.71875
1,4,A-A,201912,362857.15625
2,6,A-A,201901,220000.00000
3,6,A-A,201904,220000.00000
4,7,A-A,201903,220000.00000
...,...,...,...,...
539626,79537,E-E,201907,124385.00000
539627,79537,E-E,201905,124392.00000
539628,79537,E-E,201901,124590.00000
539629,79539,E-E,201911,120714.00000


In [34]:
pivotTarget = pd.pivot_table(targetdf2 , values = 'MontoSigno' , index = ['id'] , columns = 'Producto-Tipo' , fill_value = 0 , aggfunc = np.sum)

In [35]:
pivotTargetdf = pivotTarget.copy()
pivotTargetdf.reset_index(inplace = True)

In [36]:
pivotTargetdfid = pivotTargetdf['id']
#pivotTargetdfperiod = pivotTargetdf['Periodo']
pivotTargetdf.drop(['id'] , axis = 1 , inplace = True)

In [37]:
pivotTargetdf

Producto-Tipo,A-A,B-B,C-D,D-E,E-E
0,0.00000,-293498,1.905354e+07,1601860,501007
1,234285.71875,-120000,0.000000e+00,0,1147862
2,362857.15625,-317254,0.000000e+00,121142,742421
3,0.00000,-518916,0.000000e+00,243570,243985
4,440000.00000,0,0.000000e+00,1098462,741689
...,...,...,...,...,...
67209,0.00000,0,4.028929e+05,0,0
67210,0.00000,-369855,0.000000e+00,856382,240930
67211,0.00000,0,1.049500e+07,125658,497344
67212,0.00000,-129712,0.000000e+00,245428,0


In [38]:
for i in pivotTargetdf.columns.values:
    pivotTargetdf[i] = pivotTargetdf[i].apply(test)
pivotTargetdf['id'] = pivotTargetdfid

In [39]:
pivotTargetdf

Producto-Tipo,A-A,B-B,C-D,D-E,E-E,id
0,0,0,1,1,1,2
1,1,0,0,0,1,3
2,1,0,0,1,1,4
3,0,0,0,1,1,5
4,1,0,0,1,1,6
...,...,...,...,...,...,...
67209,0,0,1,0,0,79534
67210,0,0,0,1,1,79535
67211,0,0,1,1,1,79537
67212,0,0,0,1,0,79538


# Juntando DF

In [40]:
pivotFinal.reset_index(inplace = True)

In [41]:
#pivotFinal['Key'] = pivotFinal['id'].apply(str) + "-" + pivotFinal['Periodo'].apply(str)

In [42]:
pivotFinal

Producto-Tipo,id,A-G,A-K,A-L,C-C,D-F,E-F,F-D,F-H,F-I,F-J,G-K
0,1,4953988,-3.686182e+07,0.000000e+00,0.000000e+00,0,0,7.321078e+06,-7.561078e+06,-1.549649e+06,7.561078e+06,-5.770217e+06
1,2,8433301,-2.367750e+07,0.000000e+00,-1.605180e+07,-1699501,-384461,2.664929e+07,-1.151481e+06,-9.371714e+06,2.784929e+07,-4.938157e+07
2,3,10501637,-3.556323e+07,1.120000e+06,0.000000e+00,0,-1367568,1.802073e+06,-2.282073e+06,-6.012641e+05,2.282073e+06,-2.034374e+07
3,4,3696143,-9.662305e+06,1.697867e+05,0.000000e+00,-491115,-618740,6.297714e+07,-6.345714e+07,-1.440000e+06,6.513714e+07,-2.362493e+07
4,5,2327401,-9.319559e+06,0.000000e+00,0.000000e+00,-243570,-363838,2.811429e+06,-4.611429e+06,-4.491429e+06,4.611429e+06,-3.446029e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
79534,79535,3161029,-1.767045e+07,0.000000e+00,0.000000e+00,-854239,-240947,2.292857e+06,-2.115000e+06,-3.372857e+06,3.492857e+06,-3.040691e+07
79535,79536,3621677,-1.738624e+07,0.000000e+00,0.000000e+00,0,0,2.034100e+06,-1.539227e+06,-3.234100e+06,2.971243e+06,-3.196310e+07
79536,79537,130260,-2.513137e+05,0.000000e+00,-8.082290e+06,0,-495712,8.725352e+06,-3.198913e+06,0.000000e+00,8.965352e+06,0.000000e+00
79537,79538,3423924,-1.027632e+07,0.000000e+00,0.000000e+00,-245428,-241000,2.760000e+05,-1.588571e+06,-1.828571e+06,2.196000e+06,-7.167486e+06


In [43]:
#pivotTargetdf['Key'] = pivotTargetdf['id'].apply(str) + "-" + pivotTargetdf['Periodo'].apply(str)

# Transacciones Final

In [44]:
final = pd.merge(pivotFinal,pivotTargetdf, on = 'id' , how = 'left').fillna(0)
final

Producto-Tipo,id,A-G,A-K,A-L,C-C,D-F,E-F,F-D,F-H,F-I,F-J,G-K,A-A,B-B,C-D,D-E,E-E
0,1,4953988,-3.686182e+07,0.000000e+00,0.000000e+00,0,0,7.321078e+06,-7.561078e+06,-1.549649e+06,7.561078e+06,-5.770217e+06,0.0,0.0,0.0,0.0,0.0
1,2,8433301,-2.367750e+07,0.000000e+00,-1.605180e+07,-1699501,-384461,2.664929e+07,-1.151481e+06,-9.371714e+06,2.784929e+07,-4.938157e+07,0.0,0.0,1.0,1.0,1.0
2,3,10501637,-3.556323e+07,1.120000e+06,0.000000e+00,0,-1367568,1.802073e+06,-2.282073e+06,-6.012641e+05,2.282073e+06,-2.034374e+07,1.0,0.0,0.0,0.0,1.0
3,4,3696143,-9.662305e+06,1.697867e+05,0.000000e+00,-491115,-618740,6.297714e+07,-6.345714e+07,-1.440000e+06,6.513714e+07,-2.362493e+07,1.0,0.0,0.0,1.0,1.0
4,5,2327401,-9.319559e+06,0.000000e+00,0.000000e+00,-243570,-363838,2.811429e+06,-4.611429e+06,-4.491429e+06,4.611429e+06,-3.446029e+07,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79534,79535,3161029,-1.767045e+07,0.000000e+00,0.000000e+00,-854239,-240947,2.292857e+06,-2.115000e+06,-3.372857e+06,3.492857e+06,-3.040691e+07,0.0,0.0,0.0,1.0,1.0
79535,79536,3621677,-1.738624e+07,0.000000e+00,0.000000e+00,0,0,2.034100e+06,-1.539227e+06,-3.234100e+06,2.971243e+06,-3.196310e+07,0.0,0.0,0.0,0.0,0.0
79536,79537,130260,-2.513137e+05,0.000000e+00,-8.082290e+06,0,-495712,8.725352e+06,-3.198913e+06,0.000000e+00,8.965352e+06,0.000000e+00,0.0,0.0,1.0,1.0,1.0
79537,79538,3423924,-1.027632e+07,0.000000e+00,0.000000e+00,-245428,-241000,2.760000e+05,-1.588571e+06,-1.828571e+06,2.196000e+06,-7.167486e+06,0.0,0.0,0.0,1.0,0.0


In [45]:
final.to_pickle('TransaccionesTraining6.pkl')